### Analysis description
Here we aim at drawing fullscreen average runs on the surface of individual participant the pRF parameters.
This analysis precede the manual definition of the visual ROIs.

In [ ]:
# Imports
import os
import numpy as np
import nibabel as nb
import ipdb
import warnings
import cortex
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib as mpl
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001','sub-002','sub-003','sub-004',
            'sub-005','sub-006','sub-007','sub-008']
subjects_plot = ['sub-001','sub-002','sub-003','sub-004',
                 'sub-005','sub-006','sub-007','sub-008']
gaze_tasks = ['FullScreen','GazeCenter','GazeLeft','GazeRight']
attend_tasks = ['AttendFix','AttendBar']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
#base_dir = '/Users/martinszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
pycortex_dir = "{}/cortex".format(pp_dir)

# Plot parameters
save_svg = False # if True draw in overaly.svg
xfm_name = 'identity.fmriprep'
amp_th = 0
r2_th = [0.05, 1]
ecc_th = [0.1, 20]
sd_th = [0.1, 20]
alpha_range = [0, 0.4]
r2_range = [0, 1]
ecc_range = [0, 10]
size_range = [0, 10]
cmap_polar, cmap_uni, cmap_ecc_size = 'hsv', 'Reds', 'Spectral'
col_offset = 1.0/14.0
cmap_steps = 255

In [ ]:
# Write pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

In [ ]:
def draw_cortex(subject, xfmname, data, vmin, vmax, description, cortex_type='VolumeRGB',
                cmap='Viridis', cbar='discrete', cmap_steps=255,
                alpha=None, depth=1, thick=1, height=1024, sampler='nearest',
                with_curvature=True, with_labels=False, with_colorbar=False,
                with_borders=False, curv_brightness=0.95, curv_contrast=0.05, add_roi=False,
                roi_name='empty', col_offset=0, cbar_label=''):
    """
    Plot brain data onto a previously saved flatmap.
    Parameters
    ----------
    subject             : subject id (e.g. 'sub-001')
    xfmname             : xfm transform
    data                : the data you would like to plot on a flatmap
    cmap                : colormap that shoudl be used for plotting
    vmins               : minimal values of 1D 2D colormap [0] = 1D, [1] = 2D
    vmaxs               : minimal values of 1D/2D colormap [0] = 1D, [1] = 2D
    description         : plot title
    cortex_type         : cortex function to create the volume (VolumeRGB, Volume2D)
    cbar                : color bar layout
    cbar_label          : colorbar label
    cmap_steps          : number of colormap bins
    alpha               : alpha map
    depth               : Value between 0 and 1 for how deep to sample the surface for the flatmap (0 = gray/white matter boundary, 1 = pial surface)
    thick               : Number of layers through the cortical sheet to sample. Only applies for pixelwise = True
    height              : Height of the image to render. Automatically scales the width for the aspect of the subject's flatmap
    sampler             : Name of sampling function used to sample underlying volume data. Options include 'trilinear', 'nearest', 'lanczos'
    with_curvature      : Display the rois, labels, colorbar, annotated flatmap borders, or cross-hatch dropout?
    with_labels         : Display labels?
    with_colorbar       : Display pycortex' colorbar?
    with_borders        : Display borders?
    curv_brightness     : Mean brightness of background. 0 = black, 1 = white, intermediate values are corresponding grayscale values.
    curv_contrast       : Contrast of curvature. 1 = maximal contrast (black/white), 0 = no contrast (solid color for curvature equal to curvature_brightness).
    add_roi             : add roi -image- to overlay.svg
    roi_name            : roi name
    col_offset          : colormap offset between 0 and 1

    Returns
    -------
    braindata - pycortex volume file
    """
    import cortex
    import numpy as np
    
    # define colormap
    try: base = plt.cm.get_cmap(cmap)
    except: base = cortex.utils.get_cmap(cmap)
    
    if '_alpha' in cmap: base.colors = base.colors[1,:,:]
    val = np.linspace(0, 1,cmap_steps+1,endpoint=False)
    colmap = colors.LinearSegmentedColormap.from_list('my_colmap',base(val), N = cmap_steps)
    
    if cortex_type=='VolumeRGB':
        # convert data to RGB
        vrange = float(vmax) - float(vmin)
        norm_data = ((data-float(vmin))/vrange)*cmap_steps
        mat = colmap(norm_data.astype(int))*255.0
        alpha = alpha*255.0

        # define volume RGB
        braindata = cortex.VolumeRGB(channel1=mat[...,0].T.astype(np.uint8), channel2=mat[...,1].T.astype(np.uint8), channel3=mat[...,2].T.astype(np.uint8),
                                     alpha=alpha.T.astype(np.uint8), subject=subject, xfmname=xfmname)
    elif cortex_type=='Volume2D':
        braindata = cortex.Volume2D(dim1=data.T, dim2=alpha.T, subject=subject, xfmname=xfmname, description=description,
                                    cmap=cmap, vmin=vmin[0], vmax=vmax[0], vmin2=vmin[1], vmax2=vmax[1])
    
    braindata_fig = cortex.quickshow(braindata=braindata, depth=depth, thick=thick, height=height, sampler=sampler, with_curvature=with_curvature, with_labels=with_labels, 
                                     with_colorbar=with_colorbar, with_borders=with_borders, curvature_brightness=curv_brightness, curvature_contrast=curv_contrast)

    if cbar == 'polar':
        try: base = plt.cm.get_cmap(cmap)
        except: base = cortex.utils.get_cmap(cmap)
        val = np.arange(1,cmap_steps+1)/cmap_steps - (1/(cmap_steps*2))
        val = np.fmod(val+col_offset,1)
        colmap = colors.LinearSegmentedColormap.from_list('my_colmap', base(val), N=cmap_steps)
        cbar_axis = braindata_fig.add_axes([0.5, 0.07, 0.8, 0.2], projection='polar')
        norm = colors.Normalize(0, 2*np.pi)
        t = np.linspace(0,2*np.pi,200,endpoint=True)
        r = [0,1]
        rg, tg = np.meshgrid(r,t)
        im = cbar_axis.pcolormesh(t, r, tg.T,norm=norm, cmap=colmap)
        cbar_axis.set_yticklabels([])
        cbar_axis.set_xticklabels([])
        cbar_axis.set_theta_zero_location("W")
        cbar_axis.spines['polar'].set_visible(False)

    elif cbar == 'ecc':
        colorbar_location = [0.5, 0.07, 0.8, 0.2]
        n = 200
        cbar_axis = braindata_fig.add_axes(colorbar_location, projection='polar')
        t = np.linspace(0,2*np.pi, n)
        r = np.linspace(0,1, n)
        rg, tg = np.meshgrid(r,t)
        c = tg
        im = cbar_axis.pcolormesh(t, r, c, norm = mpl.colors.Normalize(0, 2*np.pi), cmap=colmap)
        cbar_axis.tick_params(pad=1,labelsize=15)
        cbar_axis.spines['polar'].set_visible(False)
        box = cbar_axis.get_position()
        cbar_axis.set_yticklabels([])
        cbar_axis.set_xticklabels([])
        axl = braindata_fig.add_axes([0.97*box.xmin,0.5*(box.ymin+box.ymax), box.width/600,box.height*0.5])
        axl.spines['top'].set_visible(False)
        axl.spines['right'].set_visible(False)
        axl.spines['bottom'].set_visible(False)
        axl.yaxis.set_ticks_position('right')
        axl.xaxis.set_ticks_position('none')
        axl.set_xticklabels([])
        axl.set_yticklabels(np.linspace(vmin, vmax, 3),size = 'x-large')
        axl.set_ylabel('$dva$\t\t', rotation=0, size='x-large')
        axl.yaxis.set_label_coords(box.xmax+30,0.4)
        axl.patch.set_alpha(0.5)

    elif cbar == 'discrete':
        colorbar_location= [0.8, 0.05, 0.1, 0.05]
        cmaplist = [colmap(i) for i in range(colmap.N)]
        if cortex_type == 'Volume2D':
            bounds = np.linspace(vmin[0], vmax[0], cmap_steps+1)
            bounds_label = np.linspace(vmin[0], vmax[0], 3)
        else:
            bounds = np.linspace(vmin, vmax, cmap_steps + 1)  
            bounds_label = np.linspace(vmin, vmax, 3)
        norm = mpl.colors.BoundaryNorm(bounds, colmap.N)
        
        cbar_axis = braindata_fig.add_axes(colorbar_location)
        cb = mpl.colorbar.ColorbarBase(cbar_axis, cmap=colmap, norm=norm, ticks=bounds_label, boundaries=bounds,orientation='horizontal')
        cb.set_label(cbar_label,size='x-large')

    if add_roi == True:
        cortex.utils.add_roi(data=braindata, name=roi_name, open_inkscape=False, add_path=False, depth=depth, thick=thick, sampler=sampler, with_curvature=with_curvature,
                             with_colorbar=with_colorbar, with_borders=with_borders, curvature_brightness=curv_brightness, curvature_contrast=curv_contrast)
    return braindata

In [ ]:
for subject in ['sub-001','sub-002']:
    
    # Define and create folder
    fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
    flatmaps_dir = '{}/{}/prf/pycortex/flatmaps'.format(pp_dir, subject)
    datasets_dir = '{}/{}/prf/pycortex/datasets'.format(pp_dir, subject)
    try: os.makedirs(flatmaps_dir) 
    except: pass
    try: os.makedirs(datasets_dir)
    except: pass

    # Load fit parameters
    r2_im = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,gaze_tasks[0]))
    ecc_im = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,gaze_tasks[0]))
    sd_im = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,gaze_tasks[0]))
    amp_im = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,gaze_tasks[0]))
    x_im = nb.load('{}/{}_task-{}_par-x.nii.gz'.format(fit_dir,subject,gaze_tasks[0]))
    y_im = nb.load('{}/{}_task-{}_par-y.nii.gz'.format(fit_dir,subject,gaze_tasks[0]))
    
    # Create param matrix
    r2_idx, ecc_idx, sd_idx, amp_idx, x_idx, y_idx = 0, 1, 2, 3, 4, 5
    param_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],6))*np.nan
    param_mat[...,r2_idx] = r2_im.get_fdata()
    param_mat[...,ecc_idx] = ecc_im.get_fdata()
    param_mat[...,sd_idx] = sd_im.get_fdata()
    param_mat[...,amp_idx] = amp_im.get_fdata()
    param_mat[...,x_idx] = x_im.get_fdata()
    param_mat[...,y_idx] = y_im.get_fdata()
    
    # Threshold param matrix by masking r2
    param_mat_th = param_mat
    amp_down =  param_mat_th[...,amp_idx] > amp_th
    r2_th_down, r2_th_up = param_mat_th[...,r2_idx] >= r2_th[0],  param_mat_th[...,r2_idx] <= r2_th[1]
    ecc_th_down, ecc_th_up = param_mat_th[...,ecc_idx] >= ecc_th[0], param_mat_th[...,ecc_idx] <= ecc_th[1]
    sd_th_down, sd_th_up = param_mat_th[...,sd_idx] >= sd_th[0], param_mat_th[...,sd_idx] <= sd_th[1]
    all_th = np.array((amp_down, r2_th_down, r2_th_up, ecc_th_down, ecc_th_up, sd_th_down, sd_th_up)) 
    param_mat[np.logical_and.reduce(all_th)==False, r2_idx]=0
    maps_names = []
    
    # Define pRF R2
    r2_data = param_mat[...,r2_idx]
    alpha = (r2_data - alpha_range[0])/(alpha_range[1]-alpha_range[0])
    alpha[alpha>1]=1
    param_r2 = {'data': param_mat[...,r2_idx], 'cmap': cmap_uni, 'alpha': alpha, 'vmin': r2_range[0],'vmax': r2_range[1],'cbar': 'discrete', 'cbar_label': 'R2',
                'description': '{}: pRF R2'.format(gaze_tasks[0]), 'curv_brightness': 1, 'curv_contrast': 0.1, 'add_roi': False}
    maps_names.append('r2')
    
    # Define pRF eccentricity
    ecc_data = param_mat[...,ecc_idx]
    param_ecc = {'data': ecc_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 'vmin': ecc_range[0], 'vmax': ecc_range[1],'cbar': 'ecc','roi_name': 'pRF_eccentricity',
                 'description': '{}: pRF eccentricity'.format(gaze_tasks[0]), 'curv_brightness': 1, 'curv_contrast': 0.1, 'add_roi': save_svg}
    maps_names.append('ecc')
    
    # Define pRF polar angle
    pol_comp_num = param_mat[...,x_idx] + 1j * -1*param_mat[...,y_idx] # here by mistake we flipped the y axis
    polar_ang = np.angle(pol_comp_num)
    ang_norm = (polar_ang + np.pi) / (np.pi * 2.0)
    ang_norm = np.fmod(ang_norm + col_offset,1)
    
    param_polar = { 'data': ang_norm, 'cmap': cmap_polar, 'alpha': alpha, 'vmin': 0, 'vmax': 1, 'cmap_steps': cmap_steps, 'cbar': 'polar', 'roi_name': 'pRF_polar_angle',
                    'col_offset': col_offset, 'description': '{}: pRF polar angle'.format(gaze_tasks[0]), 'curv_brightness': 0.1, 'curv_contrast': 0.25, 'add_roi': save_svg}
    maps_names.append('polar')
    
    # Define pRF size
    size_data = param_mat[...,sd_idx]
    param_size = {'data': size_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 'vmin': size_range[0], 'vmax': size_range[1], 'cbar': 'discrete', 'cbar_label': 'pRF size (dva)',
                  'description': '{}: pRF size'.format(gaze_tasks[0]), 'curv_brightness': 1, 'curv_contrast': 0.1, 'add_roi': False}
    maps_names.append('size')
    
    # Flatmaps
    volumes = {}
    for maps_name in maps_names:

        # Create flatmap
        subject_param = {'subject': subject, 'xfmname': xfm_name}
        exec('param_{}.update(subject_param)'.format(maps_name))
        exec('volume_{} = draw_cortex(**param_{})'.format(maps_name,maps_name))
        exec("plt.savefig('{}/{}_task-{}_{}.pdf')".format(flatmaps_dir, subject, gaze_tasks[0], maps_name))
        plt.show()

        # Save flatmap in dataset
        exec('vol_description = param_{}["description"]'.format(maps_name))
        exec('volume = volume_{}'.format(maps_name))
        volumes.update({vol_description:volume})

    # Save dataset
    dataset_file = "{}/{}_task-{}.hdf".format(datasets_dir, subject, gaze_tasks[0])
    if os.path.exists(dataset_file):
        print('deleting old file : {}'.format(dataset_file))
        os.remove(dataset_file)
    print('saving: {}'.format(dataset_file))
    dataset = cortex.Dataset(data = volumes)
    dataset.save(dataset_file)